<div style="text-align: center;">
    <h1> <font style="bold"> Agent-Based Model </font></h1>
    <h2><font style="bold"> General Paz </font></h2>
    <h3><font style="bold">Integrantes:</font></h3>
</div>
<div style="display: flex; justify-content: center;">
    <h4><ul>
        <li>Noguera Azul</li>
        <li>Gonzalez Rocio</li>
        <li>Guledjian Patricio</li>
        </ul>
    </h4>
</div>

## Descripción del modelo

### Objetivo

Nuestro objetivo es implementar una simulación realista de la Avenida General Paz. En los tramos de Liniers a Lugones. 

### Entidades / Agentes

Las entidades del modelo, qué serán los qué circulen por él, en nuestro modelo corresponden a los automóviles con sus respectivos conductores. Los agentes son autónomos, es decir, están dotados con comportamientos que les permiten tomar decisiones independientes.

### Atributos

Las entidades tienen distintos atributos que lo describen como:

- Posición **(x)**.
- Velocidad **(v)**.
- Longitud del auto. **(l)**.
- Tiempo de permanencia en la autopista **(t)**.
- Tiempo de reacción de frenado. **(tr)**
- Tiempo en el que el Agente arribó a la Gral. Paz. **(at)**

##### Interacción con otros agentes
Cada agente tiene un vehiculo por detras y por delante. Del vehiculo de adelante dependerá si el agente frena o no. Las reducciones de velocidad son las que causarán los *shockwaves*. 

- Distancia entre vehículo de atras **(ba)**.
- Distancia entre vehiculo de adelante **(fa)**. 

##### Comportamiento
- Los agentes reaccionan (mantienen su velocidad, aceleran y desaceleran) dependiendo de el ambiente y su vecindario. 
- Dependiendo de el comportamiento del agente que maneje el vehiculo, el mismo:
  -  respetará los limites de velocidad o no.
  -   tendrá mas o menos probabilidad de choque.


### Ambiente

El ambiente de nuestro agent-based transportation model será la infraestructura y capacidades de los nodos y enlaces de la red vial. En estas capacidades se crean efectos de congestión (velocidades de viaje reducidas) limitando el número de agentes que se desplazan a través de la red de transporte en un momento dado.

### Variables de Decisión

Las variables de decisión son las qué generarán distintas alternativas o escenarios en el modelo. Es decir, estas serán las qué ayudarán a determinar cuál es el layout óptimo para la solución. 

Nosotros planteamos la **Hipótesis** siguiente:
- *A mayor cantidad de autos autónomos, menor será la probabilidad de choques y menor el tiempo total de los agentes en recorrer la Gral. Paz*

La variable de decisión principal será entonces el porcentaje de autos autónomos en la avenida. 

### Variables de Referencia

Las variables de referencia son las obtenidas cómo resultado de la simulación. Se buscará mejorarlas en función al objetivo.
Tiempo promedio de recorrido de los vehículos.

### Variables de Estado

Las variables de estado son aquellas qué contienen la información necesaria para describir la situación actual de los recursos.
- Tasa de Ocupación en las principales zonas de aglomeración.
- Tasa de ocupación ocasionada por accidentes, vehículos detenidos, obras en la ruta, policías parando, etc.
- Tasa de ocupación ocasionada por mal tiempo.
- Tasa de ocupación del peaje o semáforos  (si es que se agregan).

### Distribuciones

#### Distribución de arribos de vehiculos

Optado por la utilización de una *distribución exponencial* como modelo para representar los arribos de vehículos, y esta elección se fundamenta en diversas consideraciones clave.

La distribución exponencial negativa es una selección frecuentemente empleada en la modelación de procesos de llegada aleatoria. En el contexto de rutas de vehículos, esta distribución se ajusta adecuadamente cuando se asume que los arribos de vehículos son eventos independientes y se producen a una tasa constante a lo largo del tiempo. En otras palabras, la probabilidad de que un vehículo llegue en un periodo determinado es constante y no depende del momento en que llegaron los vehículos previos.

Sin embargo, es fundamental destacar que emplearemos parámetros diferentes para la distribución, dependiendo de la franja horaria en la que nos encontremos, ya sea durante las horas pico o en otros momentos del día.

La distribución que elegiremos para modelar los arribos será la siguiente:
$$f(x | \lambda) = e^{(-\lambda x)}$$

$$
\lambda =
\begin{cases}
\frac{1}{\mu_1} = \frac{1}{15 \text{s}} \, & \text{si es hora pico} \\
\frac{1}{\mu_2} = \frac{1}{45 \text{s}}, & \text{si no es hora pico}
\end{cases}
$$

#### distribución de velocidad de arribos


Para modelar la velocidad de llegada de los agentes a la autopista, se optó por emplear una *distribución normal* con una media de 17 metros por segundo (61.2 kilómetros por hora) y una varianza de 1 metro por segundo (3.6 kilómetros por hora). Esta elección se basa en la búsqueda de un enfoque realista y equilibrado en la simulación.

La elección de una velocidad de llegada no extrema como media tiene como objetivo representar un escenario inicial donde los agentes ingresan gradualmente a la autopista, permitiéndoles adaptarse a las condiciones simuladas de tráfico. Esta aproximación evita situaciones iniciales de alta congestión que podrían resultar en colisiones debido a la falta de adaptación de los agentes.

Por otro lado, la selección de una varianza pequeña contribuye a limitar la dispersión en las velocidades de llegada de los agentes, lo que ayuda a mantener una transición más suave durante el proceso de entrada a la autopista. En resumen, esta elección busca proporcionar una base sólida para la simulación al garantizar que los agentes comiencen con velocidades razonables y converjan gradualmente hacia las condiciones de tráfico simuladas.

En consecuencia, el uso de una distribución normal con estos parámetros se considera adecuado para establecer una entrada controlada y realista de los agentes en la simulación, minimizando la probabilidad de choques provocados por discrepancias iniciales en las velocidades de los vehículos.


$$
f(x) = \frac{1}{\sigma \sqrt{2\pi}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}
$$

$$
\begin{align*}
\mu & = 17m/s \\
\sigma & = 1m/s
\end{align*}
$$

#### Distribución de velocidad deseada

La distribución triangular es una elección adecuada para simular la velocidad deseada de los agentes en una ruta con un mínimo de 8.34m/s (30 km/h), una moda de 19.45m/s (70 km/h) y un máximo de 27.78m/s (100 km/h) por lo siguiente:

Es una distribución continua que permite modelar situaciones en las que existe un valor mínimo, un valor máximo y un valor modal. En el contexto de la velocidad de los vehiculos en una carretera, es razonable suponer que la mayoría de los vehiculos desearán moverse a una velocidad cercana a la moda (70 km/h) y que habrá algunos que desearan ir más rápido o más lento. 

se optó por emplear esta distribución para que los agentes busquen satisfacer su deseo de acercarse a su velocidad deseada en cada instante de tiempo mientras sea posible. 



$$
f(x) = \begin{cases}
0 & \text{si } x < 30 \\
\frac{2(x-30)}{(70-30)(100-30)} & \text{si } 30 \leq x < 70 \\
\frac{2(70-x)}{(70-30)(70-100)} & \text{si } 100 \leq x \leq 70 \\
0 & \text{si } x > 100
\end{cases}
$$

#### Distribución de variación de velocidad

En nuestra simulación de la Gral. Paz, cada agente se comporta de manera única y puede variar en su velocidad debido a diferentes factores, como la relación con otros agentes y las condiciones del entorno. Para modelar esta variación de velocidad, inicialmente utilizamos la distribución triangular, que nos proporciona una velocidad modal estimada para cada agente.

Sin embargo, sabemos que los agentes no mantendrán una velocidad constante en la carretera, ya que pueden optar por no respetar las leyes de velocidad, frenar de manera abrupta o ajustar su velocidad de acuerdo con la situación. Por lo tanto, para capturar esta variabilidad en la velocidad de manera más realista, decidimos emplear la distribución normal.

La distribución normal se seleccionó como una opción apropiada para modelar la variación de velocidad de los agentes debido a sus ventajas. Esta distribución es ampliamente reconocida por su capacidad para representar la variabilidad en datos del mundo real. Además, su simetría y versatilidad permiten modelar la idea de que algunos agentes pueden conducir a velocidades más altas o más bajas que la velocidad modal, mientras que la mayoría se mantendrá cerca de esta velocidad central.

La distribución normal también se ajusta fácilmente mediante la variación de la varianza, lo que nos permite controlar el grado de variación en las velocidades de los agentes de acuerdo con las necesidades de nuestra simulación.

$$
f(x) = \frac{1}{\sigma \sqrt{2\pi}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}
$$

$$
\begin{align*}
\mu & \text{ es la velocidad del agente obtenida a partir de la distribución triangular}, \\
\sigma & = 5 \text{ km/h} \text{ (varianza)}.
\end{align*}
$$



entran todos con media 17 y desvio 1, buscan llegar a la velocidad deseada dada por la triangular 
            '''
            la distribución de variacion de velocidad viene dada por una normal con media 
            la diferencia entre la velocidad deseada y la actual multiplicado por 1/3 ponele para que sea realista
            y desvio (gio dijo 1 ) hay que ver que onda

            '''

#### Distribución de frenado

Sabemos que para que un agente logre frenar sin colisionar debe reducir su velocidad  como minimo a la de el agente de adelante. Sin embargo modelar la distribución de frenado de esta forma no se ajusta a la realidad debido a que los mismos son unicos e independientes, por lo que tienen distintos tiempos de reacción, distintas velocidades y distintas velocidades deseadas. 

Por este motivo, decidimos agregarle un factor aleatoreo que se amolde a las propiedades de cada agente.

Elegimos la *distribución exponencial* como medida de error de la reducción necesaria para evitar colisiones. Esto se debe a varias razones fundamentales:

- **Falta de Memoria a Largo Plazo:** Esta propiedad implica que la probabilidad de que un vehículo frene con fuerza en un momento dado no depende de cuánto tiempo haya transcurrido desde el último frenado. 

- **Ajuste de la "Agresividad" del Frenado:** La distribución exponencial nos brinda la flexibilidad de ajustar el grado de "agresividad" en el frenado de los vehículos a través del parámetro de tasa (λ). Al variar el valor de λ, podemos modelar diferentes comportamientos de frenado. Un valor más alto de λ indica que los vehículos son menos propensos a frenar con fuerza cuando están cerca del vehículo de adelante, lo que podría representar situaciones de tráfico más fluidas. Por otro lado, un valor más bajo de λ indica una mayor probabilidad de frenado en distancias cercanas, lo que podría reflejar situaciones de tráfico más congestionadas o conductores más cautelosos.

- **Modelización de Eventos Independientes:** La distribución exponencial es ampliamente utilizada para modelar eventos aleatorios que ocurren de manera independiente a lo largo del tiempo. En el contexto del tráfico, asumimos que los frenados de los vehículos son eventos estocásticos e independientes, ya que las decisiones de frenado de un vehículo no están influenciadas por los eventos de frenado de otros vehículos en el pasado. Esta suposición es coherente con la variabilidad inherente a las interacciones de tráfico en situaciones reales.


##### Es importante comprender como cambia la formula en los diferentes contextos...
- Cuando el agente se encuentra a menos de un segundo del de adelante:
  
    $$\text{reducción} = \frac{\text{velocidad del agente de adelante} - \text{velocidad del agente de atras}}{1 - e^{(\lambda x)} }$$

- Cuando el agente se encuentra entre uno y dos segundo del de adelante:

    $$\text{reducción} = \frac{\text{velocidad del agente de adelante} - \text{velocidad del agente de atras}}{2 - e^{(\lambda x)} }$$


##### Pero, ¿Que significa el $\lambda$?

Este parametro es el tiempo de reacción de el agente. A continuación explicaremos la distribución que utilizamos para el tiempo de reacción.

#### Distribución del tiempo de reacción del agente

Elegimos la *distribución normal* para simular el tiempo de reacción debido a su capacidad para representar la variabilidad realista en los tiempos de reacción de las personas al frenar. La distribución normal se ajusta bien a situaciones en las que la mayoría de los individuos tienen tiempos de reacción cercanos a un valor promedio (la media), con menos personas que experimentan tiempos de reacción significativamente más cortos o más largos. Esto refleja con precisión cómo la mayoría de las personas conducen con tiempos de reacción similares, pero aún existe variabilidad natural. 

Luego de una busqueda de estudios de investigación acerca del comportamiento de los conductores en carreteras, llegamos a la conclusión que el tiempo de reacción promedio de estimulos visuales es de aproximadamente *0.25*. Este puede variar dependiendo la edad, la estatura (los estímulos en las personas altas tienen que recorrer mayor distancia en el sistema nervioso), las enfermedades, el alcohol, las drogas, el cansancio, algunos medicamentos, las condiciones climatológicas, el estrés, etc. Se adjunta a continuación la pagina que tomamos de referencia:

https://sites.google.com/a/iesmonreal.es/kronosferia/3eso/tiempo-de-reaccion

Por otro lado, se decidió por un desvio pequeño ya que las diferencias de los tiempos de reacción entre los agentes es demasiado pequeña en unidad de segundo. 

$$
f(x) = \frac{1}{\sigma \sqrt{2\pi}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}
$$

$$
\begin{align*}
\mu & = 0.25s \\
\sigma & = 0.005s
\end{align*}
$$


## Simulación

In [571]:
import random 
from typing import Optional
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

import pygame
import sys

nunca da menos que la diferencia de velocidad

In [572]:
class Agent(pygame.sprite.Sprite):

    def __init__(self):
        self.position = 0                                             
        self.max_velocity = 80/3.6                                    
        self.length = 5                                              
        self.front_agent = None                                      
        self.back_agent = None                                       
        self.collision = False                                        
        self.desired_velocity = random.triangular(30/3.6, 100/3.6, 70/3.6)
        self.velocity = random.normalvariate(17, 1)
        self.time = 0
        self.arrival_time = 0
        self.reaction_time = random.normalvariate(0.25, 0.005) 
    
    def __str__(self):
        front_agent_info = (
            f"(t={self.front_agent.time}, x={self.front_agent.position}, "
            f"v={self.front_agent.velocity})"
            if self.front_agent is not None else "None"
        )
        
        back_agent_info = (
            f"(t={self.back_agent.time}, x={self.back_agent.position}, "
            f"v={self.back_agent.velocity})"
            if self.back_agent is not None else "None"
        )
        
        return (
            f"Agent(t={self.time}, at={self.arrival_time}, x={self.position}, "
            f"v={self.velocity}, rt={self.reaction_time}, "
            f"fa={front_agent_info}, ba={back_agent_info})"
        )

    def dist_to_front_agent(self):
        if self.front_agent == None:
            return None

        else: #hay otro agente adelante
            return abs(self.position - self.front_agent.position - self.length)


    def dist_to_back_agent(self):
        if self.back_agent == None:
            return None

        else: #hay otro agent atras
            return abs(self.back_agent.position - self.position - self.back_agent.length)

    def speed_variation(self):
        if self.dist_to_front_agent() is not None and self.dist_to_front_agent() / (self.velocity - self.front_agent.velocity) >= 3:
            # si se encuentra "lejos" del agente de adelante, buscará acercarse a la velocidad deseada.
            self.velocity += random.normalvariate((self.desired_velocity - self.velocity) * 1/3, 1)

        elif self.dist_to_front_agent() is not None and self.dist_to_front_agent() / (self.velocity - self.front_agent.velocity) < 3 and self.dist_to_front_agent() / (self.velocity - self.front_agent.velocity) >= 2:
            #Si se encuentra a distancia media del agente de adelante, solo puede acercarse a la velocidad deseada si es menor a la actual.
            if self.velocity > self.desired_velocity:
                self.velocity += random.normalvariate((self.front_agent.velocity - self.velocity) * 1/3, 1)



    def reduce_speed(self):
        # si se encuentra "cerca" del agente de adelante, reducirá su velocidad mediante una distribución exponencial con media 1.
        if self.dist_to_front_agent() is not None and self.dist_to_front_agent() / (self.velocity - self.front_agent.velocity) < 1:   

            reduction = (self.front_agent.velocity - self.velocity) /(1 - stats.expon.rvs(scale=self.reaction_time))

            # verificamos que sea una reducción posible para un agente humano. 
            if reduction < -4:
                #Imposible que frene un humano
                self.collision = True
    
            else:
                self.velocity += reduction

            #CHEQUEAR SI PUEDE PASAR ESTO...
            if self.velocity < 0:
                self.velocity = 0  
        

        # si se encuentra "lejos" del agente de adelante, reducirá su velocidad mediante una distribución exponencial con media 3.
        elif self.dist_to_front_agent() is not None and self.dist_to_front_agent() / (self.velocity - self.front_agent.velocity) < 2 and self.dist_to_front_agent() / (self.velocity - self.front_agent.velocity) >= 1:
            
            reduction = (self.front_agent.velocity - self.velocity) /(2 - stats.expon.rvs(scale=self.reaction_time))

            # verificamos que sea una reducción posible para un agente humano. 
            if reduction < -4:
                #Imposible que frene un humano
                self.collision = True
                
            else:
                self.velocity += reduction

            #CHEQUEAR SI PUEDE PASAR ESTO...
            if self.velocity < 0:
                self.velocity = 0
    

    def move(self):
        self.position = self.position + self.velocity * 1 
    
    def get_position(self):
        return self.position

    def update(self):
        self.move()
        self.reduce_speed()
        self.speed_variation()

        self.time += 1

In [573]:
class Road:
    
    def __init__(self, m:int, c:bool):
        self.km = m
        self.agents = []
        self.time = 0
        self.total_time = 100
        self.congestion = c
        self.collision_count = 0
    
    def add_agent(self, rate):
        # Si no hay agentes en la Gral.Paz, agregamos el primer agente
        if not self.agents:
            self.agents.append(Agent())
            #######print("Primer agente agregado: " + str(self.agents[0]))
            
            self.time_to_next_arrival = stats.expon.rvs(scale=1/rate)

        else:
            # Verificamos el tiempo desde el último agente
            if self.time - self.agents[-1].arrival_time > self.time_to_next_arrival:
                # Agregamos un nuevo agente a la lista
                new_agent = Agent()
                self.agents.append(new_agent)
                
                new_agent.arrival_time = self.time
                self.time_to_next_arrival = stats.expon.rvs(scale=1/rate)
                #######print("Agregando agente nro. ", new_agent.arrival_time)

                # Configuramos el front y back car para el nuevo agente
                if len(self.agents) >= 2:
                    position_new_agent = self.agents.index(new_agent)
                    new_agent.front_agent = self.agents[position_new_agent - 1]
                    self.agents[position_new_agent - 1].back_agent = new_agent

        self.update_road()

    def time_lapse(self):
        while self.time < self.total_time:
            if self.congestion == True:
                # Agrega autos con distribución exponencial negativa con media = 15
                self.add_agent(10)
            else:
                # Agrega autos con distribución exponencial negativa con media = 45
                self.add_agent(30)

        
    
    def update_road(self):
        self.time += 1

        removed = set()
        for agent in self.agents:
            agent.update()
            print("Agente:", self.agents.index(agent), "->", agent)
            
            #Si el agente llega al final de la carretera lo eliminamos
            if agent.get_position() > self.km:
                removed.add(agent)

            #Reasignamos el front y back car de los agentes extremos de la lista
            if len(self.agents) > 0:
                self.agents[0].front_agent = None

            if agent.collision == True:
                removed.add(agent)
                removed.add(agent.front_agent)
                self.collision_count += 1

        # reasingo la posiciones de la lista
        for agent in removed:
            # Si hay choque lo eliminamos y reasignamos los front y back car
            self.agents.remove(agent)
        
        for agent in self.agents:
            if self.agents.index(agent) != 0:
                        agent.front_agent = self.agents[self.agents.index(agent) - 1]
            else:
                agent.front_agent = None
                
            if self.agents.index(agent) != (len(self.agents) - 1):
                agent.back_agent = self.agents[self.agents.index(agent) + 1]
            else:
                agent.back_agent = None

        self.time_lapse()

In [574]:
road = Road(m=17000, c=True)
road.time_lapse()

Agente: 0 -> Agent(t=1, at=0, x=17.640374965638557, v=17.640374965638557, rt=0.25396117534483387, fa=None, ba=None)
Agente: 0 -> Agent(t=2, at=0, x=35.280749931277114, v=17.640374965638557, rt=0.25396117534483387, fa=None, ba=(t=0, x=0, v=15.641318611626337))
Agente: 1 -> Agent(t=1, at=1, x=15.641318611626337, v=16.813015436159073, rt=0.24312690701772435, fa=(t=2, x=35.280749931277114, v=17.640374965638557), ba=None)
Agente: 0 -> Agent(t=3, at=0, x=52.92112489691567, v=17.640374965638557, rt=0.25396117534483387, fa=None, ba=(t=1, x=15.641318611626337, v=16.813015436159073))
Agente: 1 -> Agent(t=2, at=1, x=32.45433404778541, v=18.89618227242688, rt=0.24312690701772435, fa=(t=3, x=52.92112489691567, v=17.640374965638557), ba=(t=0, x=0, v=18.658035114286367))
Agente: 2 -> Agent(t=1, at=2, x=18.658035114286367, v=19.07426747530161, rt=0.24882316347248584, fa=(t=2, x=32.45433404778541, v=18.89618227242688), ba=None)
Agente: 0 -> Agent(t=4, at=0, x=70.56149986255423, v=17.640374965638557, rt

In [575]:
import pygame
import sys
import time

# Inicializar Pygame
pygame.init()

# Configuración de la ventana
width, height = 1000, 200
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Avenida General Paz")

# Colores
white = (255, 255, 255)
gray = (200, 200, 200)
green = (0, 150, 0)
black = (0, 0, 0)

# Crear una carretera
general_paz = road

delay = 100  # Milisegundos de retraso entre fotogramas (ajusta según sea necesario)

# Bucle principal
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Agregar un retraso para ralentizar la simulación
    pygame.time.delay(delay)

    # Actualizar la carretera y los agentes en cada iteración
    general_paz.update_road()

    # Dibujar el pasto verde arriba de la carretera
    pygame.draw.rect(screen, green, (0, 0, width, height // 3))

    # Carretera gris
    pygame.draw.rect(screen, gray, (0, height // 3, width, height // 3))

    # Dibujar el pasto verde abajo de la carretera
    pygame.draw.rect(screen, green, (0, 2 * height // 3, width, height // 3))

    # Líneas blancas en el medio de la carretera
    line_width = 10
    line_height = 5
    num_lines = 30
    gap = width // (num_lines + 1)
    for i in range(num_lines):
        x = (i + 1) * gap - line_width // 2
        pygame.draw.rect(screen, white, (x, height // 2 - line_height // 2, line_width, line_height))

    # Dibujar los agentes en la carretera
    for agent in general_paz.agents:
        pygame.draw.rect(screen, black, (agent.get_position(), height // 3 - 10, agent.length, 20))


    pygame.display.flip()

# Salir del juego
pygame.quit()
sys.exit()


Agente: 0 -> Agent(t=28, at=0, x=463.09315900195605, v=16.539041392926997, rt=0.24326974997380932, fa=None, ba=(t=16, x=397.88123036770855, v=18.860256695868287))
Agente: 1 -> Agent(t=17, at=84, x=416.74148706357687, v=20.602350972436312, rt=0.2511049616235168, fa=(t=28, x=463.09315900195605, v=16.539041392926997), ba=(t=13, x=323.17347909087505, v=24.085877164573137))
Agente: 2 -> Agent(t=14, at=87, x=347.2593562554482, v=21.91645136106078, rt=0.2460626300141017, fa=(t=17, x=416.74148706357687, v=20.602350972436312), ba=(t=12, x=296.0810515239564, v=22.61813204075727))
Agente: 3 -> Agent(t=13, at=88, x=318.6991835647136, v=20.863537803858655, rt=0.24683234406242444, fa=(t=14, x=347.2593562554482, v=21.91645136106078), ba=(t=11, x=259.7345141447127, v=23.104013137855))
Agente: 4 -> Agent(t=12, at=89, x=282.8385272825677, v=22.55201505692504, rt=0.24990801029948284, fa=(t=13, x=318.6991835647136, v=20.863537803858655), ba=(t=10, x=208.17744837989204, v=19.113368940088062))
Agente: 5 -> 

KeyboardInterrupt: 